<a href="https://colab.research.google.com/github/cristianmejia00/clustering/blob/main/Topic_Models_using_BERTopic_TOPIC_MODEL_20241101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with BERTopic

🔴 copied from the [Kubota Colab](https://colab.research.google.com/drive/1YsDp5_qGXGJKsEXsS8DO8CA_lqZc6EpA).  

`Topic Models` are methods to automatically organize a corpus of text into topics.

Topic Model process:
1. Data preparation
2. Tranform text to numeric vectors
3. Multidimensionality reduction
4. Clustering
5. Topic analysis
6. Cluster assignation


This notebook uses the library `BERTopic` which is a one-stop solution for topic modeling including handy functions for plotting and analysis. However, BERTopic does not have a function to extract the X and Y coords from UMAP. If we need the coordinates then use the notebooks `Topic_Models_using_Transformers` instead. In any other situation, when a quick analysis is needed this notebook may be better.

This notebook is also the one needed for the heatmap codes included in this folder.

`BERTopic` is Python library that handles steps 2 to 6.
BERT topic models use the transformer architechture to generate the embeds (i.e. the vector or numeric representation of words) and are currently the state-of-the-art method for vectorization.

This notebook shows how to use it.

---
Reading:
[Topic Modeling with Deep Learning Using Python BERTopic](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504)
[Advanced Topic Modeling with BERTopic](https://www.pinecone.io/learn/bertopic/)


# Requirements

## Packages installation and initialization

In [ ]:
!pip install bertopic[visualization]

In [ ]:
import pandas as pd
import numpy as np
import time
import math
import uuid
import re
import os
import json
import pickle
from datetime import date
from itertools import compress
from bertopic import BERTopic
from umap import UMAP
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

## Connect your Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
def find_e_keys(dictionary):
    # List comprehension to find keys starting with 'e'
    e_keys = [key for key in dictionary if str(key).lower().startswith('e')]
    return e_keys

# 🔴 Input files and options

Go to your Google Drive and create a folder in the root directory. We are going to save all related data in that directory.
Upload the dataset of news into the above folder.
- The dataset should be a `.csv` file.
- Every row in the dataset is a document
- It can any kind of columns. Some columns must contain the text we want to analyze. For example, a dataset of academic articles may contain a "Title" and/or "Abstract" column.

In [ ]:
# The bibliometrics folder
# Colab
ROOT_FOLDER_PATH = "drive/MyDrive/Bibliometrics_Drive"

# Mac
ROOT_FOLDER_PATH = "/Users/cristian/Library/CloudStorage/GoogleDrive-cristianmejia00@gmail.com/My Drive/Bibliometrics_Drive"

# Change to the name of the folder where the dataset is uploaded inside the above folder
project_folder = 'Q327 TI Policy'

analysis_id = 'a01_cn__f01_dc__c01_lv'

# Filtered label
settings_directive = "settings_analysis_directive_2025-04-17-23-20.json"

In [ ]:
# Read settings
with open(f'{ROOT_FOLDER_PATH}/{project_folder}/{analysis_id}/{settings_directive}', 'r') as file:
    settings = json.load(file)

In [ ]:
# Input dataset
dataset_file_path = f"{ROOT_FOLDER_PATH}/{settings['metadata']['project_folder']}/{settings['metadata']['filtered_folder']}/dataset_raw_cleaned.csv"

In [ ]:
# Function to save files
def save_as_csv(df, save_name_without_extension, with_index):
    "usage: `save_as_csv(dataframe, 'filename')`"
    df.to_csv(f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv", index=with_index)
    print("===\nSaved: ", f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv")

In [ ]:
# prompt: a function to save object to a pickle file
def save_object_as_pickle(obj, filename):
  """
  Saves an object as a pickle file.

  Args:
      obj: The object to be saved.
      filename: The filename of the pickle file.
  """
  with open(filename, "wb") as f:
    pickle.dump(obj, f)


In [ ]:
# prompt: a function to load pickle object given a path
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [ ]:
# Open the data file
df = pd.read_csv(f"{dataset_file_path}", encoding='latin-1')
print(df.shape)
df.head()



---



## PART 2: Topic Model

In [ ]:
# bibliometrics_folder
# project_folder
# project_name_suffix
# ROOT_FOLDER_PATH = f"drive/MyDrive/{bibliometrics_folder}"

#############################################################
# Embeddings folder
embeddings_folder_name = settings['tmo']['embeds_folder']

# Which column has the year of the documents?
my_year = settings['tmo']['year_column']

# Number of topics. Select the number of topics to extract.
# Choose 0, for automatic detection.
n_topics = settings['tmo']['n_topics']

# Minimum number of documents per topic
min_topic_size = settings['tmo']['min_topic_size']

In [ ]:
# Get the embeddings back.
embeddings = load_pickle(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/embeddings.pck")
corpus =     pd.read_csv(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/corpus.csv").reset_index(drop=True)

In [ ]:
# Combine embeddings
documents = corpus.text.to_list()

In [ ]:
# corpus['uuid'] = [uuid.uuid4() for _ in range(len(corpus.index))]
# corpus['X_N'] = [i for i in range(1, len(corpus.index)+1)]

In [ ]:
len(documents)

In [ ]:
#len(embeddings) == len(documents)
len(embeddings['embeddings']) == len(documents)

In [ ]:
from hdbscan.hdbscan_ import HDBSCAN
# Execute the topic model.
# I suggest changing the values marked with #<---
# The others are the default values and they'll work fine in most cases.
# This will take several minutes to finish.

# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

if n_topics == 0:
  # Initiate topic model with HDBScan (Automatic topic selection)
  topic_model_params = HDBSCAN(min_cluster_size=min_topic_size,
                               metric='euclidean',
                               cluster_selection_method='eom',
                               prediction_data=True)
else:
  # Initiate topic model with K-means (Manual topic selection)
  topic_model_params = KMeans(n_clusters = n_topics)

# Initiate BERTopic
topic_model = BERTopic(umap_model = umap_model,
                       hdbscan_model = topic_model_params,
                       min_topic_size=min_topic_size,
                       #nr_topics=15,          #<--- Footnote 1
                       n_gram_range=(1,3),
                       language='english',
                       calculate_probabilities=True,
                       verbose=True)



# Footnote 1: This controls the number of topics we want AFTER clustering.
# Add a hashtag at the beggining to use the number of topics returned by the topic model.
# When using HDBScan nr_topics will be obtained after orphans removal, and there is no warranty that `nr_topics < HDBScan topics`.
# thus, with HDBScan `nr_topics` means N topics OR LESS.
# When using KMeans nr_topics can be used to further reduce the number of topics.
# We use the topics as returned by the topic model. So we do not need to activate it here.

In [ ]:
# Compute topic model
#topics, probabilities = topic_model.fit_transform(documents, embeddings)
topics, probabilities = topic_model.fit_transform(documents, embeddings['embeddings'])

In [ ]:
# Get the list of topics
# Topic = the topic number. From the largest topic.
#         "-1" is the generic topic. Genericr keywords are aggegrated here.
# Count = Documents assigned to this topic
# Name = Top 4 words of the topic based on probability
# Representation = The list of words representing this topic
# Representative_Docs = Documents assigned to this topic
tm_summary = topic_model.get_topic_info()
tm_summary

In [ ]:
# Save the topic model assets
tm_folder_path = f'{ROOT_FOLDER_PATH}/{project_folder}/{settings["metadata"]["analysis_id"]}'

if not os.path.exists(tm_folder_path):
  !mkdir $tm_folder_path

tm_summary.to_csv(f'{tm_folder_path}/topic_model_info.csv', index=False)

In [ ]:
# Number of topics found
found_topics = max(tm_summary.Topic) + 1
found_topics

In [ ]:
# Confirm all documents are assigned
sum(tm_summary.Count) == len(corpus)

In [ ]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

In [ ]:
# Get the top 10 documents for a topic
topic_model.get_representative_docs(0)

In [ ]:
# Others

# # Get the number of documents per topic (same as in the table above)
# topic_model.get_topic_freq(0)

# # Get the main keywords per topic
# topic_model.get_topics()

In [ ]:
# Print the parameters used. (For reporting)
topic_model.get_params()

In [ ]:
tm_params = dict(topic_model.get_params())
for key, value in tm_params.items():
    tm_params[key]=  str(value)
with open(f'{tm_folder_path}/topic_model_params.json', 'w') as f:
    json.dump(tm_params, f, ensure_ascii=False, indent=4)
    print('Done')

In [ ]:
# Get the topic score for each paper and its assigned topic
topic_distr, _ = topic_model.approximate_distribution(documents, batch_size=1000)
distributions = [distr[topic] if topic != -1 else 0 for topic, distr in zip(topics, topic_distr)]

In [ ]:
topic_distr

In [ ]:
# Document information. Including the topic assignation
dataset_clustering_results = topic_model.get_document_info(documents, df = corpus, metadata={"Score": distributions})

# Standar format for report analysis
dataset_clustering_results = dataset_clustering_results.drop(columns=['text'])
dataset_clustering_results['X_E'] = dataset_clustering_results['Score']
dataset_clustering_results['X_C'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['level0'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['cl99'] = False
dataset_clustering_results['cl-99'] = False
dataset_clustering_results.head()

In [ ]:
# Save the dataframe
dataset_clustering_results.to_csv(f'{tm_folder_path}/dataset_minimal.csv', index=False)

In [ ]:
# Save the topic model
topic_model.save(f'{tm_folder_path}/topic_model_object.pck')



---

